In [72]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.28.0 to work with mlops


In [73]:
import os
from pathlib import Path

# Get experiment folder
experiment_folder = Path(os.getcwd()).parent
print(experiment_folder)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-mlops/code/Users/s147056/image-restoration


In [74]:
model = ws.models['image_resto']
print(model.name, 'version', model.version)

image_resto version 3


## Create yaml for env

In [75]:
from azureml.core.conda_dependencies import CondaDependencies 

# Print the .yml file
env_file = os.path.join(experiment_folder, "src", "deploy", "image_resto_env.yml")
with open(env_file,"r") as f:
    print(f.read())         

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

  - click
  - Sphinx
  - coverage
  - awscli
  - flake8
  - python-dotenv>=0.5.1
  - wandb
  - pathlib2
  - argparse
  - torch
  - torchvision
  - opencv-python
  - joblib
  - -e git+https://github.com/heshe/image-restoration
channels:
- anaconda
- conda-forge



## Deploy

In [76]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Environment, Model

env = Environment.from_pip_requirements('image_resto_env', os.path.join(experiment_folder, 'requirements.txt'))

# Set path for scoring script
script_file = os.path.join(experiment_folder, "src", "deploy", "score.py")

# Configure the scoring environment
inference_config = InferenceConfig(entry_script=script_file,
                                   environment=env)



deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "image-reconstruction-service"
if service_name in ws.webservices:
    ws.webservices[service_name].delete()


service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-16 09:07:41+00:00 Creating Container Registry if not exists.
2021-06-16 09:07:42+00:00 Building image..
2021-06-16 09:18:17+00:00 Generating deployment configuration.
2021-06-16 09:18:18+00:00 Submitting deployment to compute..
2021-06-16 09:18:24+00:00 Checking the status of deployment image-reconstruction-service..
2021-06-16 09:21:48+00:00 Checking the status of inference endpoint image-reconstruction-service.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e1ba8f4a-0b37-4fb5-8c70-018259b5f460
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, AttributeError: module 'src.models.model_FC' has no attribute 'Net', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, AttributeError: module 'src.models.model_FC' has no attribute 'Net', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e1ba8f4a-0b37-4fb5-8c70-018259b5f460
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, AttributeError: module 'src.models.model_FC' has no attribute 'Net', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, AttributeError: module 'src.models.model_FC' has no attribute 'Net', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: e1ba8f4a-0b37-4fb5-8c70-018259b5f460\nMore information can be found using

In [ ]:
import json
import torch
import matplotlib.pyplot as plt

x_new = torch.rand(1, 224, 224)*255

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
reconstruction = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
reconstruction = json.loads(reconstruction)

%matplotlib inline

type(reconstruction)